# biobert ner

In [79]:
import tensorflow as tf
tf.float32

tf.float32

In [51]:
import os
import sys
from sagemaker import get_execution_role
# import time

role = get_execution_role()

# notebooks_dir = '../notebooks'
# working_dir = '../'
# if working_dir not in sys.path:
#     sys.path.append(working_dir)
    
# DATA_DIR_BIOBERT = '../data/download/finetuned_biobert_model'
# # import run_ner
# # from run_ner import BC5CDRProcessor, model_fn_builder, file_based_input_fn_builder, filed_based_convert_examples_to_features, result_to_pair



hyperparameters = {'task_name': 'bc5cdr', 'do_train': 'true', 'do_eval': 'true', 'do_predict': 'false', 'do_export': 'true', 
                   'data_dir': '/opt/ml/input/data/training', 'output_dir': '/opt/ml/model', 
                   'vocab_file': 'vocab.txt', 'horovod':'true',
                   'bert_config_file': 'bert_config.json', 
                   'max_seq_length': 128, 'train_batch_size': 64, 'eval_batch_size':1,'num_train_epochs': 1}

In [52]:
# import sagemaker

# inputs = sagemaker.Session().upload_data(path='/home/ec2-user/SageMaker/NvidiaDeepLearningExamples/TensorFlow/LanguageModeling/BERT/data/data/', key_prefix='biobertner')
# print(inputs)

# import subprocess

# train_instance_type='local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
# train_instance_type = 'local_gpu'
    
# print("Train instance type = " + train_instance_type)

train_instance_type='ml.g4dn.xlarge'

# byos

In [53]:
from sagemaker.tensorflow import TensorFlow
import datetime
distributions = {"mpi": {"enabled": True}}
TrainingJobName = 'biobertTrain'+ datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
estimator = TensorFlow(entry_point='run_ner.py',
                       source_dir='.',
                       train_instance_type=train_instance_type, 
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.15.2',
                       volume_size=100,
                       TrainingJobName=TrainingJobName,
                       py_version='py37',
                       distributions=distributions,
                       script_mode=True)

estimator.fit({'training': 's3://sagemaker-us-east-2-310850127430/biobertner/'})

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-09-06 03:08:35 Starting - Starting the training job...ProfilerReport-1630897715: InProgress
...
2021-09-06 03:09:22 Starting - Launching requested ML instances......
2021-09-06 03:10:22 Starting - Preparing the instances for training......
2021-09-06 03:11:31 Downloading - Downloading input data
2021-09-06 03:11:31 Training - Downloading the training image......
2021-09-06 03:12:34 Training - Training image download completed. Training in progress..WARNING:tensorflow:From /usr/local/lib/python3.7/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2021-09-06 03:12:37,185 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-09-06 03:12:37,538 sagemaker-training-toolkit INFO     Module run_ner.py does not provide a setup.py. 
Generating setup.py
2021-09-06 03:12:37,539 sagemaker-training-toolkit INFO     Generating setup.cfg
2021-09-06 03:12:37,53

# 部署

In [54]:
# training_job_1_details["AlgorithmSpecification"]["TrainingImage"]
#763104351884.dkr.ecr.us-east-2.amazonaws.com/tensorflow-inference:


In [55]:
# byos
infer_image = '763104351884.dkr.ecr.us-east-2.amazonaws.com/tensorflow-inference:1.15.2-gpu'
TrainingJobName='tensorflow-training-2021-09-05-08-39-27-084'
instance_type = 'ml.m5.xlarge'
import boto3
from sagemaker import get_execution_role
import datetime
import sagemaker

client = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()

model_1_name = "biobert" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
training_job_1_details = client.describe_training_job(TrainingJobName=TrainingJobName)

# create model
create_model_1_response = client.create_model(
    ModelName=model_1_name,
    PrimaryContainer={
        "Image": infer_image,
        "Mode": "SingleModel",
        "ModelDataUrl": training_job_1_details["ModelArtifacts"]["S3ModelArtifacts"],
        "Environment": {
            "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
            "SAGEMAKER_PROGRAM": training_job_1_details["HyperParameters"]["sagemaker_program"],
            "SAGEMAKER_REGION": region,
            "SAGEMAKER_SUBMIT_DIRECTORY": training_job_1_details["HyperParameters"][
                "sagemaker_submit_directory"
            ],
        },
    },
    ExecutionRoleArn=get_execution_role(),
)

# endpoint_config
endpoint_config_1_name = "biobert" + datetime.datetime.now().strftime(
    "%Y-%m-%d-%H-%M-%S"
)

endpoint_config_1_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_1_name,
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": model_1_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.xlarge",
            "InitialVariantWeight": 1,
        },
    ],
)

endpoint_config_1_response



{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:310850127430:endpoint-config/biobert2021-09-06-03-26-00',
 'ResponseMetadata': {'RequestId': 'bfe1b7ff-cf14-4d6e-bd6a-b5853fb6b502',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bfe1b7ff-cf14-4d6e-bd6a-b5853fb6b502',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '107',
   'date': 'Mon, 06 Sep 2021 03:26:00 GMT'},
  'RetryAttempts': 0}}

In [ ]:
endpoint_name = "biobert" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_1_name,
)

create_endpoint_response
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


# infer

In [80]:
import re
import os
import pickle
import tokenization
import numpy as np

text= """
The authors describe the case of a 56 year old woman with chronic, severe heart failure
secondary to dilated cardiomyopathy and absence of significant ventricular arrhythmias
who developed QT prolongation and torsade de pointes ventricular tachycardia during one cycle
of intermittent low dose (2.5 mcg/kg per min) dobutamine.
This report of torsade de pointes ventricular tachycardia during intermittent dobutamine
supports the hypothesis that unpredictable fatal arrhythmias may occur even with low doses
and in patients with no history of significant rhythm disturbances.
The mechanisms of proarrhythmic effects of Dubutamine are discussed.
"""
hyperparameters = {'task_name': 'bc5cdr', 'do_train': 'false', 'do_eval': 'false', 'do_predict': 'true', 'do_export': 'false', 
                   'data_dir': '/opt/ml/input/data/training', 'output_dir': '/opt/ml/model', 
                   'vocab_file': 'vocab.txt', 'horovod':'true',
                   'bert_config_file': 'bert_config.json', 
                   'max_seq_length': 128, 'train_batch_size': 64, 'eval_batch_size':1,'num_train_epochs': 1}





def seg_char(sent):
    """
    把句子按字分开，不破坏英文结构
    """
    # 首先分割 英文 以及英文和标点
    pattern_char_1 = re.compile(r'([\W])')
    parts = pattern_char_1.split(sent)
    parts = [p for p in parts if len(p.strip())>0]

    return parts

max_seq_length = hyperparameters['max_seq_length']
vocab_file = hyperparameters['vocab_file']


tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=True)

with open('label2id.pkl', 'rb') as rf:  # TODO need use label2id.pkl in model.tar.gz
    label2id = pickle.load(rf)
    id2label = {value: key for key, value in label2id.items()}
    
tokens = ['[CLS]']
tokens.extend(seg_char(text)[:max_seq_length-2])
tokens.append('[SEP]')
print(tokens)
with open('label2id.pkl', 'rb') as rf:  # TODO need use label2id.pkl in model.tar.gz
    label2id = pickle.load(rf)
    id2label = {value: key for key, value in label2id.items()}
print(label2id)
print(id2label)

['[CLS]', 'The', 'authors', 'describe', 'the', 'case', 'of', 'a', '56', 'year', 'old', 'woman', 'with', 'chronic', ',', 'severe', 'heart', 'failure', 'secondary', 'to', 'dilated', 'cardiomyopathy', 'and', 'absence', 'of', 'significant', 'ventricular', 'arrhythmias', 'who', 'developed', 'QT', 'prolongation', 'and', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'one', 'cycle', 'of', 'intermittent', 'low', 'dose', '(', '2', '.', '5', 'mcg', '/', 'kg', 'per', 'min', ')', 'dobutamine', '.', 'This', 'report', 'of', 'torsade', 'de', 'pointes', 'ventricular', 'tachycardia', 'during', 'intermittent', 'dobutamine', 'supports', 'the', 'hypothesis', 'that', 'unpredictable', 'fatal', 'arrhythmias', 'may', 'occur', 'even', 'with', 'low', 'doses', 'and', 'in', 'patients', 'with', 'no', 'history', 'of', 'significant', 'rhythm', 'disturbances', '.', 'The', 'mechanisms', 'of', 'proarrhythmic', 'effects', 'of', 'Dubutamine', 'are', 'discussed', '.', '[SEP]']
{'B': 1, 'I': 2, 'O': 3,

In [81]:
# %load_ext autoreload

from infer_preprocess import BC5CDRProcessor,filed_based_convert_examples_to_features
import tokenization

do_lower_case=True
# tokenization.validate_case_matches_checkpoint(do_lower_case, init_checkpoint)

# Create the tokenizer.
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

# Use the data processor for BC5CDR
processor = BC5CDRProcessor()

predict_examples = processor.get_test_examples('./')

label_list = processor.get_labels()

In [ ]:
predict_file = "predict.tf_record"
import collections

features=filed_based_convert_examples_to_features(predict_examples, label_list,
                                                 max_seq_length, tokenizer,
                                                 predict_file, mode="test")

In [82]:
input_ids=[101,18834,7277,7934,11937,11714,11522,2401,2076,23852,2079,8569,15464,3170,6753,1996,10744,2008,21446,10611,12098,25032,22123,26837,3022,2089,5258,2130,2007,2659,21656,1998,1999,5022,2007,2053,2381,1997,3278,6348,24535,1012,1996,102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
input_mask=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
segment_ids=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
label_ids=[5,3,4,4,3,4,4,4,3,3,3,4,4,4,3,3,3,3,3,3,3,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
payload={
    'input_ids':input_ids,
    'input_mask':input_mask,
    'segment_ids':segment_ids,
    'label_ids':label_ids
}

In [88]:
# runtime = boto3.client("sagemaker")

# response = runtime.describe_endpoint(
#     EndpointName='biobert2021-09-05-11-48-04'
# )
# response

payload = {'instances': [{'input_ids': input_ids
                      }]}

'''
,
                      'input_mask': input_mask,
                       'segment_ids': segment_ids,
                       'label_ids': label_ids
'''

"\n,\n                      'input_mask': input_mask,\n                       'segment_ids': segment_ids,\n                       'label_ids': label_ids\n"

In [100]:
import boto3
runtime = boto3.client("sagemaker-runtime")

# input_ids = np.array(input_ids, dtype=np.int64)
# segment_ids = np.array(segment_ids, dtype=np.int64)
# input_mask = np.array(input_mask, dtype=np.int64)   
# label_ids = np.array(label_ids, dtype=np.int64)   

# payload = np.concatenate([np.expand_dims(input_ids, axis=0), np.expand_dims(segment_ids, axis=0), np.expand_dims(input_mask, axis=0)])

response = runtime.invoke_endpoint(EndpointName='biobert2021-09-06-02-22-37',
                                       ContentType='application/json',
                                       Body=json.dumps(payload))
response = eval(response['Body'].read().decode('utf-8'))
response



{'predictions': [[5,
   3,
   4,
   4,
   3,
   4,
   4,
   4,
   3,
   3,
   3,
   4,
   4,
   4,
   4,
   3,
   3,
   3,
   4,
   3,
   3,
   4,
   4,
   4,
   4,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   4,
   4,
   3,
   3,
   6,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]]}

In [94]:
result_label = [id2label[id] for id in response['predictions'][0] if id != 0]
result_label

['[CLS]',
 'O',
 'X',
 'X',
 'O',
 'X',
 'X',
 'X',
 'O',
 'O',
 'O',
 'X',
 'X',
 'X',
 'X',
 'O',
 'O',
 'O',
 'X',
 'O',
 'O',
 'X',
 'X',
 'X',
 'X',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'X',
 'X',
 'O',
 'O',
 '[SEP]']

In [101]:
for i in range(len(result_label)):
    ne = tokens[i]
    nes[ne] = result_label[i]
nes

{'[CLS]': '[CLS]',
 'The': 'O',
 'authors': 'X',
 'describe': 'X',
 'the': 'O',
 'case': 'X',
 'of': 'O',
 'a': 'X',
 '56': 'O',
 'year': 'O',
 'old': 'O',
 'woman': 'X',
 'with': 'X',
 'chronic': 'X',
 ',': 'X',
 'severe': 'O',
 'heart': 'O',
 'failure': 'O',
 'secondary': 'X',
 'to': 'O',
 'dilated': 'O',
 'cardiomyopathy': 'X',
 'and': 'O',
 'absence': 'X',
 'significant': 'O',
 'ventricular': 'O',
 'arrhythmias': 'O',
 'who': 'O',
 'developed': 'O',
 'QT': 'O',
 'prolongation': 'O',
 'torsade': 'O',
 'de': 'O',
 'pointes': 'O',
 'tachycardia': 'O',
 'during': 'O',
 'one': 'X',
 'cycle': 'X',
 'intermittent': 'O',
 'low': '[SEP]'}

In [98]:
ne

'low'

In [95]:
nes = {}
ne = ''
nep = ''
for i in range(len(result_label)):
    if result_label[i].startswith('B-'):
        ne = tokens[i]
        nep = result_label[i].replace('B-', '')
    elif result_label[i].startswith('I-'):
        ne += tokens[i]
    else:
        if ne != '':
            nes[ne] = nep
            ne = ''
            nep = ''
print(nes)

{}


In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm
# # Convert the text into the IOB tags format seen during training, using dummy placeholder labels
# import spacy
# nlp = spacy.load("en_core_web_sm")

# text = text.strip()
# doc = nlp(text)
# input_file = 'input.tsv'
# with open(os.path.join(input_file), 'w') as wf: 
#     for word in doc:
#         if word.text is '\n':
#             continue
#         wf.write(word.text + '\tO\n')
#     wf.write('\n') # Indicate end of text
